In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [4]:
train_dir = "D:/4th 1st semester/computer vision/tasks/Alzheimer_s Dataset/Alzheimer_s Dataset/Alzheimer_s Dataset/train"
test_dir = "D:/4th 1st semester/computer vision/tasks/Alzheimer_s Dataset/Alzheimer_s Dataset/Alzheimer_s Dataset/test"

In [5]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [6]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [7]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [8]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [9]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\Computec\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\Computec\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step


In [11]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step


In [12]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3227378	total: 127ms	remaining: 2m 6s
1:	learn: 1.2675078	total: 146ms	remaining: 1m 12s
2:	learn: 1.2213815	total: 167ms	remaining: 55.6s
3:	learn: 1.1825985	total: 187ms	remaining: 46.5s
4:	learn: 1.1474740	total: 205ms	remaining: 40.9s
5:	learn: 1.1188563	total: 223ms	remaining: 36.9s
6:	learn: 1.0933439	total: 239ms	remaining: 33.9s
7:	learn: 1.0720891	total: 265ms	remaining: 32.9s
8:	learn: 1.0527804	total: 287ms	remaining: 31.6s
9:	learn: 1.0376039	total: 306ms	remaining: 30.3s
10:	learn: 1.0209574	total: 319ms	remaining: 28.7s
11:	learn: 1.0067936	total: 334ms	remaining: 27.5s
12:	learn: 0.9929400	total: 347ms	remaining: 26.4s
13:	learn: 0.9796815	total: 362ms	remaining: 25.5s
14:	learn: 0.9688912	total: 375ms	remaining: 24.6s
15:	learn: 0.9588614	total: 389ms	remaining: 23.9s
16:	learn: 0.9494965	total: 403ms	remaining: 23.3s
17:	learn: 0.9402732	total: 416ms	remaining: 22.7s
18:	learn: 0.9317368	total: 429ms	remaining: 22.2s
19:	learn: 0.9243808	total: 443ms	remain

In [13]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [14]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 53.71%


In [15]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [16]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [19]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model("D:/4th 1st semester/computer vision/tasks/task4/task4.2/cnn_feature_extractor6.h5")
catboost_classifier = joblib.load("D:/4th 1st semester/computer vision/tasks/task4/task4.2/catboost_classifier.pkl")  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = "D:/4th 1st semester/computer vision/tasks/Alzheimer_s Dataset/Alzheimer_s Dataset/Alzheimer_s Dataset/test/NonDemented/26 (69).jpg"
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
The image is classified as: NonDemented


C:\Users\Computec\AppData\Local\Temp\ipykernel_1444\3775078136.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
